## Facebook posts - Topic Modelling

#### Dataset

- Posts made within the campaign period (Feb 12 - May 13, 2019) are taken from senatorial candidates' facebook page 
- Posts are translated to English
- Posts containing words pertaining to election campaign activities are removed to focus on posts related to platforms and the like.

In [1]:
import pandas as pd
import nltk; nltk.download('stopwords')

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
#warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to C:\Users\Arjoselle
[nltk_data]     Ortiz\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1. Import data

In [2]:
df = pd.read_csv('fb_posts_translated.csv')

#### a. Drop irrelevant columns.

In [3]:
df.drop(columns=['Unnamed: 0'], inplace=True)
df.head()

,Candidate,Translated_text
0,Bong Go,Happy Mother's Day From Tatay Digong!
1,Bong Go,Thank you so much to all of you!
2,Bong Go,"KUYA BONG GO, SATURDAY HOMETOWN DAVAO CITY FOR..."
3,Bong Go,Vhong Navarro has shown support for Kuya Bong ...
4,Bong Go,"Bong Go at Cogon Public Market, Cagayan de Oro..."


In [4]:
df.shape

(2664, 2)

In [5]:
pprint(df.iloc[0,1])

"Happy Mother's Day From Tatay Digong!"


#### b. Drop rows related to election campaign activities to focus on posts related to platforms and the like.

In [6]:
df.Translated_text = df.Translated_text.apply(lambda x: np.NaN if 'vote' in str(x).lower() else x)
df.Translated_text = df.Translated_text.apply(lambda x: np.NaN if 'thank' in str(x).lower() else x)
df.Translated_text = df.Translated_text.apply(lambda x: np.NaN if 'election' in str(x).lower() else x)
df.Translated_text = df.Translated_text.apply(lambda x: np.NaN if 'elections' in str(x).lower() else x)
df.Translated_text = df.Translated_text.apply(lambda x: np.NaN if 'senatorial' in str(x).lower() else x)
df.Translated_text = df.Translated_text.apply(lambda x: np.NaN if 'senator' in str(x).lower() else x)
df.Translated_text = df.Translated_text.apply(lambda x: np.NaN if 'ballot' in str(x).lower() else x)
df.Translated_text = df.Translated_text.apply(lambda x: np.NaN if 'candidate' in str(x).lower() else x)
df.Translated_text = df.Translated_text.apply(lambda x: np.NaN if 'campaign' in str(x).lower() else x)
df.Translated_text = df.Translated_text.apply(lambda x: np.NaN if 'rally' in str(x).lower() else x)
df.Translated_text = df.Translated_text.apply(lambda x: np.NaN if 'senado' in str(x).lower() else x)
df.Translated_text = df.Translated_text.apply(lambda x: np.NaN if 'balota' in str(x).lower() else x)

df.dropna(subset=['Translated_text'], inplace=True)
df.shape

(861, 2)

In [7]:
df.drop_duplicates(inplace=True)

### 2. Pre-process Data

In [8]:
# Convert to list
data = df.Translated_text.values.tolist()
len(data)

839

#### a. Prepare stopwords

In [9]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['thank', 'support', 'much', 'election', 'ballot', 'candidate', 'vote', 'campaign', 'ate', 'gracepoe', 'ating', 'cheldiokno', 'aking', 'nating', 'pampanga', 'watch', 'ng', 'avance', 'atty', 'live' ])

#### b. Tokenize words and Clean-up text

In [10]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))
data_words[0]

['happy', 'mother', 'day', 'from', 'tatay', 'digong']

#### c. Creating Bigram and Trigram Models

In [11]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['happy', 'mother', 'day', 'from', 'tatay', 'digong']


#### d. Remove Stopwords, Make Bigrams and Lemmatize

In [12]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
        if 'nate' in texts_out:
            print(sent)
    return texts_out

In [13]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ','VERB', 'ADV'])

print(data_lemmatized[:1])

[[]]


#### e. Create the Dictionary and Corpus needed for Topic Modeling

In [14]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[]]


In [15]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[]]

### 3. Build the Model

Using Mallet LDA

In [16]:
import os
#C:\Users\Arjoselle Ortiz\Documents\DS Classes\Eskwelabs\Capstone\mallet-2.0.8\mallet-2.0.8\bin
os.environ.update({'MALLET_HOME':r'C:/new_mallet/mallet-2.0.8/'})
mallet_path = 'C:\\new_mallet\\mallet-2.0.8\\bin\\mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=9, id2word=id2word, random_seed=123)

In [17]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(0,
  [('time', 0.04173106646058733),
   ('today', 0.023183925811437404),
   ('long', 0.023183925811437404),
   ('poor', 0.023183925811437404),
   ('water', 0.02009273570324575),
   ('official', 0.02009273570324575),
   ('system', 0.017001545595054096),
   ('countryman', 0.015455950540958269),
   ('woman', 0.013910355486862442),
   ('disaster', 0.013910355486862442)]),
 (1,
  [('country', 0.08494783904619971),
   ('public', 0.040238450074515646),
   ('medicine', 0.02533532041728763),
   ('back', 0.022354694485842028),
   ('law', 0.022354694485842028),
   ('pay', 0.01788375558867362),
   ('company', 0.01788375558867362),
   ('war', 0.013412816691505217),
   ('incentive', 0.013412816691505217),
   ('list', 0.011922503725782414)]),
 (2,
  [('good', 0.08229813664596274),
   ('price', 0.04813664596273292),
   ('market', 0.04813664596273292),
   ('sell', 0.029503105590062112),
   ('year', 0.029503105590062112),
   ('talk', 0.027950310559006212),
   ('farmer', 0.018633540372670808),
   ('tow

### 3. Tune the Model

#### a. Optimal number of topics

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start, step):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        print(num_topics)
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word,
                                                random_seed=123)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

start1 = 4 
limit1 = 13
step1 = 1

model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=start1, limit=limit1, step=step1)

x = range(start1, limit1, step1)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

4
5
6
7
8
9
10


In [ ]:
# Print the coherence scores
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

We'll choose 9 topics as it has high coherence score and 
seems a reasonable number of topics to have.

#### b. Build optimal model

In [ ]:
optimal_model = model_list[-4] # Number of Topics = 9

model_topics = optimal_model.show_topics(formatted=False)
pprint(optimal_model.print_topics(num_words=10))

### 5. Evaluate Results

#### a. Visualize the topics-keywords

In [ ]:
# Convert LDA Mallet to normal LDA since pyLDAvis only takes the latter for visualization
optimal_model2 = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(optimal_model)

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(optimal_model2, corpus, id2word)
vis

#### b. Dominant topic and its percentage contribution in each document

In [ ]:
list(optimal_model.read_doctopics)

In [ ]:
def format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)

In [ ]:
df_dominant_topic['Dominant_Topic'].value_counts()

In [ ]:
df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 0].sort_values('Topic_Perc_Contrib', ascending=False).iloc[:50]
# df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 1].sort_values('Topic_Perc_Contrib', ascending=False).iloc[:50]
# df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 2].sort_values('Topic_Perc_Contrib', ascending=False).iloc[:50]
# df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 3].sort_values('Topic_Perc_Contrib', ascending=False).iloc[:50]
# df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 4].sort_values('Topic_Perc_Contrib', ascending=False).iloc[:50]
# df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 5].sort_values('Topic_Perc_Contrib', ascending=False).iloc[:50]
# df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 6].sort_values('Topic_Perc_Contrib', ascending=False).iloc[:50]
# df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 7].sort_values('Topic_Perc_Contrib', ascending=False).iloc[:50]
# df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 7].sort_values('Topic_Perc_Contrib', ascending=False).iloc[:50]

In [ ]:
#df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 0]['Text'].values # Addressing various issues - 261
# df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 1]['Text'].values # Medicine, War on Drugs, Foreign Investments - 66
# df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 2]['Text'].values # Price of goods - 79
# df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 3]['Text'].values # Hardworking Filipinos - 92
# df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 4]['Text'].values # Jobs/Labor - 76
# df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 5]['Text'].values # Health, Youth - 83
# df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 6]['Text'].values # Good Governance, Local Government programs - 69
# df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 7]['Text'].values # Small entrepreneurs/Workers - 49
# df_dominant_topic[df_dominant_topic['Dominant_Topic'] == 8]['Text'].values # Sovereignity/Independence - 64

#### c. The most representative sentence for each topic

In [ ]:
 #Group top 5 sentences under each topic
sent_topics_sorteddf_mallet = pd.DataFrame()

sent_topics_outdf_grpd = df_topic_sents_keywords.groupby('Dominant_Topic')

for i, grp in sent_topics_outdf_grpd:
    sent_topics_sorteddf_mallet = pd.concat([sent_topics_sorteddf_mallet, 
                                             grp.sort_values(['Perc_Contribution'], ascending=[0]).head(1)], 
                                            axis=0)

# Reset Index    
sent_topics_sorteddf_mallet.reset_index(drop=True, inplace=True)

# Format
sent_topics_sorteddf_mallet.columns = ['Topic_Num', "Topic_Perc_Contrib", "Keywords", "Text"]

# Show
sent_topics_sorteddf_mallet.head(10)

#### d. Topic distribution

In [ ]:
# Number of Documents for Each Topic
topic_counts = df_topic_sents_keywords['Dominant_Topic'].value_counts()

# Percentage of Documents for Each Topic
topic_contribution = round(topic_counts/topic_counts.sum(), 4)

# Topic Number and Keywords
topic_num_keywords = df_topic_sents_keywords[['Dominant_Topic', 'Topic_Keywords']]

# Concatenate Column wise
df_dominant_topics = pd.concat([topic_num_keywords, topic_counts, topic_contribution], axis=1)

# Change Column names
df_dominant_topics.columns = ['Dominant_Topic', 'Topic_Keywords', 'Num_Documents', 'Perc_Documents']

# Show
df_dominant_topics

#### e. Top words per topic

In [ ]:
# Wordcloud of Top N words in each topic
from matplotlib import pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.colors as mcolors

cols = [color for name, color in mcolors.TABLEAU_COLORS.items()]  # more colors: 'mcolors.XKCD_COLORS'

cloud = WordCloud(stopwords=stop_words,
                  background_color='white',
                  width=2500,
                  height=1800,
                  max_words=20,
                  colormap='tab10',
                  color_func=lambda *args, **kwargs: cols[i],
                  prefer_horizontal=1.0)

topics = optimal_model.show_topics(formatted=False, num_words=15)

fig, axes = plt.subplots(3, 3, figsize=(15,15), sharex=True, sharey=True)

for i, ax in enumerate(axes.flatten()):
    fig.add_subplot(ax)
    topic_words = dict(topics[i][1])
    cloud.generate_from_frequencies(topic_words, max_font_size=300)
    plt.gca().imshow(cloud)
    plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
    plt.gca().axis('off')


plt.subplots_adjust(wspace=0, hspace=0)
plt.axis('off')
plt.margins(x=0, y=0)
plt.tight_layout()
plt.show()

In [ ]:
from sklearn.manifold import TSNE
tsne_model = TSNE(n_components=2, verbose=1, random_state=7, angle=.99, init='pca')# 13-D -> 2-D
tsne_lda = tsne_model.fit_transform(list(optimal_model.load_document_topics())) # doc_topic is document-topic matrix from LDA or GuidedLDA 

### Save Results

In [ ]:
get_document_topics = optimal_model2.get_document_topics(corpus, minimum_probability=0.0)

In [ ]:
len(get_document_topics)

In [ ]:
topic0 = []
topic1 = []
topic2 = []
topic3 = []
topic4 = []
topic5 = []
topic6 = []
topic7 = []
topic8 = []

for score in get_document_topics:
    for col in score:
        if col[0] == 0:
            topic0.append(col[1])
        elif col[0] == 1:
            topic1.append(col[1])
        elif col[0] == 2:
            topic2.append(col[1])
        elif col[0] == 3:
            topic3.append(col[1])
        elif col[0] == 4:
            topic4.append(col[1])
        elif col[0] == 5:
            topic5.append(col[1])
        elif col[0] == 6:
            topic6.append(col[1])
        elif col[0] == 7:
            topic7.append(col[1])
        elif col[0] == 8:
            topic8.append(col[1])
        elif col[0] == 9:
            topic9.append(col[1])
            
            
print(len(topic0), len(topic1), len(topic2))
print(len(topic3), len(topic4), len(topic5))
print(len(topic6), len(topic7), len(topic8))

In [ ]:
df['Topic 0'] = topic0
df['Topic 1'] = topic1
df['Topic 2'] = topic2
df['Topic 3'] = topic3
df['Topic 4'] = topic4
df['Topic 5'] = topic5
df['Topic 6'] = topic6
df['Topic 7'] = topic7
df['Topic 8'] = topic8

In [ ]:
df['Dominant Topic'] = df_dominant_topic['Dominant_Topic'].values
df

In [ ]:
#df.to_csv('fb_topic_model_9topics.csv')

### Predict topic for all posts

In [ ]:
# df_test = pd.read_csv('fb_posts_translated.csv')
# df_test.drop(columns=['Unnamed: 0'], inplace=True)
# df_test.head()

In [ ]:
# df_test.Translated_text = df.Translated_text.apply(lambda x: np.NaN if 'vote' in str(x).lower() else x)
# df_test.Translated_text = df.Translated_text.apply(lambda x: np.NaN if 'thank' in str(x).lower() else x)
# df_test.Translated_text = df.Translated_text.apply(lambda x: np.NaN if 'election' in str(x).lower() else x)
# df_test.Translated_text = df.Translated_text.apply(lambda x: np.NaN if 'elections' in str(x).lower() else x)
# df_test.Translated_text = df.Translated_text.apply(lambda x: np.NaN if 'senatorial' in str(x).lower() else x)
# df_test.Translated_text = df.Translated_text.apply(lambda x: np.NaN if 'senator' in str(x).lower() else x)
# df_test.Translated_text = df.Translated_text.apply(lambda x: np.NaN if 'ballot' in str(x).lower() else x)
# df_test.Translated_text = df.Translated_text.apply(lambda x: np.NaN if 'candidate' in str(x).lower() else x)
# df_test.Translated_text = df.Translated_text.apply(lambda x: np.NaN if 'campaign' in str(x).lower() else x)
# df_test.Translated_text = df.Translated_text.apply(lambda x: np.NaN if 'rally' in str(x).lower() else x)
# df_test.Translated_text = df.Translated_text.apply(lambda x: np.NaN if 'senado' in str(x).lower() else x)
# df_test.Translated_text = df.Translated_text.apply(lambda x: np.NaN if 'balota' in str(x).lower() else x)
# df_test.dropna(subset=['Translated_text'], inplace=True)
# df_test.shape

In [ ]:
# df_test.drop_duplicates(inplace=True)

In [ ]:
# # Convert to list
# data = df_test.Translated_text.iloc[500:1000].values.tolist()
# len(data)

In [ ]:
# data_words = list(sent_to_words(data))
# data_words[0]

In [ ]:
# # Build the bigram and trigram models
# bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
# trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# # Faster way to get a sentence clubbed as a trigram/bigram
# bigram_mod = gensim.models.phrases.Phraser(bigram)
# trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
# # Remove Stop Words
# data_words_nostops = remove_stopwords(data_words)

# # Form Bigrams
# data_words_bigrams = make_bigrams(data_words_nostops)

# # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# # python3 -m spacy download en
# nlp = spacy.load('en', disable=['parser', 'ner'])

# # Do lemmatization keeping only noun, adj, vb, adv
# data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ','VERB', 'ADV'])

# print(data_lemmatized[:1])

In [ ]:
# # Create Dictionary
# id2word = corpora.Dictionary(data_lemmatized)

# # Create Corpus
# texts = data_lemmatized

# # Term Document Frequency
# corpus = [id2word.doc2bow(text) for text in texts]

# # View
# print(corpus[:1])

In [ ]:
# predicted_results = pd.DataFrame(columns=['Topic0','Topic1','Topic2','Topic3','Topic4','Topic5','Topic6','Topic7','Topic8'])

# for index, text in enumerate(data):
#     x = optimal_model[corpus[index]]
#     predicted_results.loc[text]=x

# predicted_results

In [ ]:
predicted_results[:20]